## 1) Importing the libraries

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
df = pd.read_csv("dataset4.csv")

df.head()

,feature1,feature2,class
0,1.400628,-1.062722,1
1,1.572726,1.030960,0
2,-0.397654,-0.564270,2
3,-0.489704,1.137081,2
4,1.761848,1.191341,0


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   feature1  500 non-null    float64
 1   feature2  500 non-null    float64
 2   class     500 non-null    int64  
dtypes: float64(2), int64(1)
memory usage: 11.8 KB


In [18]:
## creating another variable to save the features except target column
features = df.drop('class', axis=1)
features

## naming the response variable as target
target = df['class']
target

0      1
1      0
2      2
3      2
4      0
      ..
495    1
496    1
497    0
498    0
499    1
Name: class, Length: 500, dtype: int64

## 2) Pipeline to implement a scaler and MLP classifier

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(features, target, test_size=0.25, random_state=42)
print("Size of training set: {} size of test set: {}".format(X_train.shape[0], X_test.shape[0]))

#  MLP classifier with different configuration settings for dataset
clfs = [
MLPClassifier(hidden_layer_sizes=(2,),learning_rate='adaptive'
                   , max_iter=3000),
MLPClassifier(hidden_layer_sizes=(2,2),learning_rate='adaptive'
                   , max_iter=1000),
MLPClassifier(hidden_layer_sizes=(5,),learning_rate='adaptive'
                   , max_iter=1000),
MLPClassifier(hidden_layer_sizes=(5,2),learning_rate='adaptive'
                   , max_iter=1000),
MLPClassifier(hidden_layer_sizes=(5,5),learning_rate='adaptive'
                   , max_iter=1000),
MLPClassifier(hidden_layer_sizes=(5,10),learning_rate='adaptive'
                   , max_iter=1000),
MLPClassifier(hidden_layer_sizes=(10,),learning_rate='adaptive'
                   , max_iter=1000),
MLPClassifier(hidden_layer_sizes=(10,2),learning_rate='adaptive'
                   , max_iter=1000),
MLPClassifier(hidden_layer_sizes=(10,5),learning_rate='adaptive'
                   , max_iter=1000),
MLPClassifier(hidden_layer_sizes=(10,10),learning_rate='adaptive'
                   , max_iter=1000)
                         ] 

from sklearn.pipeline import Pipeline
# list of classifier names
names = ["ann_1","ann_2","ann_3","ann_4","ann_5","ann_6","ann_7","ann_8","ann_9","ann_10"]

# list of scores from the models
scores = []

pipes = []  

best_score = 0

# pipeline to implement a scaler and MLP classifier and loop to find the best parameter
for clf in clfs:
    pipe = Pipeline(steps=[("scaling", StandardScaler()),('classifier', clf)
    ])
    pipe.fit(X_train, Y_train)
    pipes.append(pipe) # add the pipe so that we can use later
    Y_pred = pipe.predict(X_test)
    print(clf)
    score = pipe.score(X_test, Y_test)
    scores.append(score)
    print("model score: %.3f" % score)
    print("---------- \n")
    Y_pred = pipe.predict(X_test)
    if score > best_score:
            best_score = score
            best_parameters = clf
            
print("Best score for R2: {:.2f}".format(best_score))
# printing the best parameter
print("Best parameters: {}".format(best_parameters))


tuned_y_pred = best_parameters.predict(X_test)


Size of training set: 375 size of test set: 125
MLPClassifier(hidden_layer_sizes=(2,), learning_rate='adaptive', max_iter=3000)
model score: 0.920
---------- 

MLPClassifier(hidden_layer_sizes=(2, 2), learning_rate='adaptive',
              max_iter=1000)
model score: 0.656
---------- 

MLPClassifier(hidden_layer_sizes=(5,), learning_rate='adaptive', max_iter=1000)
model score: 0.928
---------- 

MLPClassifier(hidden_layer_sizes=(5, 2), learning_rate='adaptive',
              max_iter=1000)
model score: 0.920
---------- 

MLPClassifier(hidden_layer_sizes=(5, 5), learning_rate='adaptive',
              max_iter=1000)
model score: 0.936
---------- 

MLPClassifier(hidden_layer_sizes=(5, 10), learning_rate='adaptive',
              max_iter=1000)
model score: 0.920
---------- 

MLPClassifier(hidden_layer_sizes=(10,), learning_rate='adaptive', max_iter=1000)
model score: 0.928
---------- 

MLPClassifier(hidden_layer_sizes=(10, 2), learning_rate='adaptive',
              max_iter=1000)
model

## 3) Confusin Matrix

In [20]:
confusion_matrix = confusion_matrix(Y_test, tuned_y_pred)
print(confusion_matrix)

[[46  0  2]
 [ 7 30  0]
 [ 1  0 39]]


From our confusion matrix, we can see that for class 0, 46 predictions, which are predicted correctly and we have (30+0+39+0), 69 predictions where model correctly predicts the negative classes. We can see that for class 1, 30 predictions, which are predicted correctly and we have (46+2+39+1), 88 predictions where model correctly predicts the negative classes. Fro class 2, 39 predictions, which are predicted correctly and we have (46+7+30+0), 83 predictions where model correctly predicts the negative classes.

## 4) Classification Report

In [21]:
print(classification_report(Y_test, tuned_y_pred, target_names=['class 0', 'class 1', 'class 2']))


              precision    recall  f1-score   support

     class 0       0.85      0.96      0.90        48
     class 1       1.00      0.81      0.90        37
     class 2       0.95      0.97      0.96        40

    accuracy                           0.92       125
   macro avg       0.93      0.91      0.92       125
weighted avg       0.93      0.92      0.92       125



From here, Performance for class 0 is better with regard to recall, which is 96%, performance for class 1 is better with regard to precision, which is 100% respectively in our case. Moreover, performance for class 3 is better with regards to f1-score, which is 96%. Also, our accuracy is 91%, which means our model is 92% accurate.